# Ноутбук с моделью

## Импортируем необходимые пакеты

In [2]:
import pandas as pd

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

## Считываем данные в dataframe

In [3]:
df = pd.read_csv('data/homework.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/homework.csv'

## Подготовка данных

### Обработка пропусков в данных

In [ ]:
# Удаляем ненужные/лишние колонки 

columns_to_drop = [
    'id',
    'url',
    'region',
    'region_url',
    'price',
    'manufacturer',
    'image_url',
    'description',
    'posting_date',
    'lat',
    'long'
]

df.drop(columns_to_drop, axis=1, inplace=True)

In [ ]:
# Есть ли пропуски?

df.isnull().sum().sort_values()

state               0
price_category      0
year               36
odometer           42
transmission       45
fuel               63
model             128
title_status      166
dtype: int64

In [ ]:
# Сохраним в переменную numerical имена всех числовых признаков нашего датасета
numerical = df.select_dtypes(include=['int64', 'float64']).columns

# Сохраним в переменную categorical имена всех категориальных признаков нашего датасета
categorical = df.select_dtypes(include=['object']).columns

# В категориальных фичах заменяем пропуски модой
for feat in categorical:
    df[feat].fillna(df[feat].mode()[0], inplace=True)
    
# В численных фичах заменяем пропуски медианой
for feat in numerical:
    df[feat].fillna(df[feat].median(), inplace=True)

# Убедимся, что пропущенных значений больше нет
df.isnull().sum()

year              0
model             0
fuel              0
odometer          0
title_status      0
transmission      0
state             0
price_category    0
dtype: int64

### Удаление выбросов

In [ ]:
df['year'].describe()

count    10000.00000
mean      2010.93540
std          9.65541
min       1915.00000
25%       2008.00000
50%       2013.00000
75%       2017.00000
max       2022.00000
Name: year, dtype: float64

In [ ]:
def calculate_outliers(data):
    q25 = data.quantile(0.25)
    q75 = data.quantile(0.75)
    iqr = q75 - q25
    boundaries = (q25 - 1.5 * iqr, q75 + 1.5 * iqr)
    
    return boundaries

boundaries = calculate_outliers(df['year'])
boundaries

(1994.5, 2030.5)

In [ ]:
df.loc[df['year'] < boundaries[0], 'year'] = round(boundaries[0])
df.loc[df['year'] > boundaries[1], 'year'] = round(boundaries[1])

df['year'].describe()

count    10000.000000
mean      2011.674200
std          6.505727
min       1994.000000
25%       2008.000000
50%       2013.000000
75%       2017.000000
max       2022.000000
Name: year, dtype: float64

## Feature engineering


In [ ]:
def short_model(x):
    if not pd.isna(x):
        return x.lower().split(' ')[0]
    else:
        return x

# Добавляем фичу "short_model" – это первое слово из колонки model
df.loc[:, 'short_model'] = df['model'].apply(short_model)

# Добавляем фичу "age_category" (категория возраста)
df.loc[:, 'age_category'] =  df['year'].apply(lambda x: 'new' if x > 2013 else ('old' if x < 2006 else 'average'))

In [ ]:
df.shape

(10000, 10)

In [ ]:
# Кодируем категориальные фичи с помощью OneHotEncoder

columns_to_encode = [
    'fuel', 
    'title_status', 
    'transmission', 
    'state', 
    'short_model', 
    'age_category'
]

encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
encoder.fit(df[columns_to_encode])

df.loc[:, encoder.get_feature_names()] = encoder.transform(df[columns_to_encode])
df.head()

,year,model,fuel,odometer,title_status,transmission,state,price_category,short_model,age_category,...,x4_yukon,x4_z,x4_z3,x4_z4,x4_zephyr,x4_zx2,x4_♿,x5_average,x5_new,x5_old
0,2020.0,2500 crew cab big horn,diesel,27442.0,clean,other,tn,high,2500,new,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2016.0,explorer 4wd 4dr xlt,gas,60023.0,clean,automatic,nj,medium,explorer,new,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2017.0,golf r hatchback,gas,14048.0,clean,other,ca,high,golf,new,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,2013.0,rav4,gas,117291.0,clean,automatic,nc,medium,rav4,average,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,2021.0,altima,gas,8000.0,clean,automatic,ny,medium,altima,new,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
# Масштабируем числовые фичи

scaler = StandardScaler()

df['odometer_std'] = scaler.fit_transform(df[['odometer']])

In [ ]:
df.head()

,year,model,fuel,odometer,title_status,transmission,state,price_category,short_model,age_category,...,x4_z,x4_z3,x4_z4,x4_zephyr,x4_zx2,x4_♿,x5_average,x5_new,x5_old,odometer_std
0,2020.0,2500 crew cab big horn,diesel,27442.0,clean,other,tn,high,2500,new,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.044178
1,2016.0,explorer 4wd 4dr xlt,gas,60023.0,clean,automatic,nj,medium,explorer,new,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.527649
2,2017.0,golf r hatchback,gas,14048.0,clean,other,ca,high,golf,new,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.256522
3,2013.0,rav4,gas,117291.0,clean,automatic,nc,medium,rav4,average,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.380259
4,2021.0,altima,gas,8000.0,clean,automatic,ny,medium,altima,new,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.352405


In [ ]:
columns_to_drop = [
    'year',
    'model',
    'fuel', 
    'odometer', 
    'title_status', 
    'transmission', 
    'state', 
    'short_model', 
    'age_category'
]

df.drop(columns_to_drop, axis=1, inplace=True)

In [ ]:
df.head()

,price_category,x0_diesel,x0_electric,x0_gas,x0_hybrid,x0_other,x1_clean,x1_lien,x1_missing,x1_parts only,...,x4_z,x4_z3,x4_z4,x4_zephyr,x4_zx2,x4_♿,x5_average,x5_new,x5_old,odometer_std
0,high,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.044178
1,medium,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.527649
2,high,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.256522
3,medium,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.380259
4,medium,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.352405


## Моделирование

In [ ]:
X = df.drop(['price_category'], axis=1)
y = df['price_category']

models = [
    LogisticRegression(solver='liblinear'),
    RandomForestClassifier(),
    SVC()
]

for m in models:
    score = cross_val_score(m, X, y, cv=4, scoring='accuracy')
    print(f'model: {type(m).__name__}, acc_mean: {score.mean():.4f}, acc_std: {score.std():.4f}')

model: LogisticRegression, acc_mean: 0.7401, acc_std: 0.0076
model: RandomForestClassifier, acc_mean: 0.7559, acc_std: 0.0068
model: SVC, acc_mean: 0.7611, acc_std: 0.0073
